## Creating a function decorator 

In [2]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        x = f(*args, **kwargs)
        return ascii(x)
    return wrap

## Using the decorator

In [3]:
def rupee():
    return '₹₹°'

print(rupee())

₹₹°


In [6]:
@escape_unicode
def rupee():
    return '₹₹°'

print(rupee())

'\u20b9\u20b9\xb0'


## Creating class decorators
A decorator is a callable that takes in an callable as an argument and returns a callable.  
Therefore to be available to be used with class it must define \__call\__ method

In [12]:
class CallCount:
    def __init__(self, f):
        self.f = f
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.f(*args, **kwargs)

In [13]:
@CallCount
def hello(name):
    print('Hello ', name, '!')

In [14]:
hello('hiro')

Hello  hiro !


In [15]:
hello('tatsuya')

Hello  tatsuya !


In [16]:
hello('aang')

Hello  aang !


In [17]:
hello.count

3

In [18]:
type(hello)

__main__.CallCount

## Class instances as decorators

In [19]:
class Trace:
    def __init__(self):
        self.enabled = True
    
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
        return wrap

In [26]:
tracer = Trace()

@tracer
def rotate_list(l):
    return l[1:] + [l[0]]

In [27]:
rotate_list([1, 2, 3])

Calling <function rotate_list at 0x7f8268fdeae8>


[2, 3, 1]

In [28]:
rotate_list([4, 5 ,6])

Calling <function rotate_list at 0x7f8268fdeae8>


[5, 6, 4]

In [31]:
tracer.enabled = False

In [35]:
rotate_list([3, 4, 1])

[4, 1, 3]

## Using multiple decorators on functions

In [38]:
@tracer
@escape_unicode
def rupee_maker(money):
    return '₹' + str(money)

In [39]:
rupee_maker(12)

Calling <function escape_unicode.<locals>.wrap at 0x7f8268770510>


"'\\u20b912'"

In [41]:
rupee_maker(23)

Calling <function escape_unicode.<locals>.wrap at 0x7f8268770510>


"'\\u20b923'"

## Using decorators on class

In [49]:
class MoneyMaker:
    def __init__(self, prefix):
        self.prefix = prefix
    @tracer
    def make_money(self, value):
        return self.prefix + str(value)

In [50]:
mm = MoneyMaker('₹')

In [51]:
mm.make_money(400)

Calling <function MoneyMaker.make_money at 0x7f82686fe8c8>


'₹400'

In [52]:
mm = MoneyMaker('$')

In [53]:
mm.make_money(1000)

Calling <function MoneyMaker.make_money at 0x7f82686fe8c8>


'$1000'

## Preserving Metadata of decorated functions

In [54]:
def noop(f):
    def noop_wrapper():
        return f()
    return noop_wrapper

In [55]:
@noop
def hello():
    '''A nice docstring'''
    print('Hello, World!')

In [56]:
hello()

Hello, World!


In [58]:
help(hello) # Losing metadata

Help on function noop_wrapper in module __main__:

noop_wrapper()



In [59]:
hello.__name__

'noop_wrapper'

In [60]:
hello.__doc__

### A simple solution

In [81]:
def noop(f):
    def noop_wrapper():
        return f()
    
    noop_wrapper.__name__ = f.__name__
    noop_wrapper.__doc__ = f.__doc__
    
    return noop_wrapper

In [82]:
def hello():
    '''A simple print function'''
    return 'Hello from {} function, I am {}'.format(hello.__name__, hello.__doc__)

In [83]:
hello()

'Hello from hello function, I am A simple print function'

In [86]:
@noop
def hello():
    '''A simple print function'''
    return 'Hello from {} function, I am {}'.format(hello.__name__, hello.__doc__)

In [87]:
hello()

'Hello from hello function, I am A simple print function'

In [89]:
hello.__name__

'hello'

### Or a more elengant way using functools

In [91]:
from functools import wraps

In [103]:
def noop(f):
   
    @wraps(f)
    def noop_wrapper():
        return f()
    
    return noop_wrapper

In [108]:
@noop
def jello():
    '''I print jello'''
    return 'jello from {} and {}'.format(jello.__name__, jello.__doc__)

In [109]:
jello()

'jello from jello and I print jello'

## Another example

In [113]:
def check_no_neg(index):
    def validator(f):
        def wrap(*args):
            if args[index] < 0:
                raise ValueError('Argument {} must be non-negative'.format(index))
            return f(*args)
        return wrap
    return validator

In [114]:
@check_no_neg(1)
def create_list(value, size):
    return [value] * size

In [116]:
create_list(1, 3)

[1, 1, 1]

In [119]:
try:
    create_list(1, -1)
except ValueError:
    print('Size argument is negative')

Size argument is negative
